In [ ]:
import os
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch import utils, cuda, device, from_numpy, backends, manual_seed, no_grad, save, float32, long, max
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [ ]:
n_epochs = 300
batch_size_train = 20
batch_size_test = 100
learning_rate = 0.01
momentum = 0.5
log_interval = 10
hl = 10
random_seed = 1
backends.cudnn.enabled = False
manual_seed(random_seed)

## Obtaining the data

In [ ]:
iris = load_iris()
X = iris.data 
y = iris.target
target_names = iris['target_names']
feature_names = iris['feature_names']
print(X.shape)
print(y.shape)
print(target_names)
print(feature_names)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

## Save the data

In [ ]:
try:
    os.mkdir("data")
except OSError:
    print("Results folder exists")
np.save('data/iris_train_data', X_train)
np.save('data/iris_train_target', y_train)
np.save('data/iris_test_data', X_test)
np.save('data/iris_test_target', y_test)

In [ ]:
X_train_loaded = np.load('data/iris_train_data.npy')
y_train_loaded = np.load('data/iris_train_target.npy')
X_test_loaded = np.load('data/iris_test_data.npy')
y_test_loaded = np.load('data/iris_test_target.npy')
print(X_train_loaded.shape)
print(y_train_loaded.shape)
print(X_test_loaded.shape)
print(y_test_loaded.shape)

In [ ]:
class IrisDataset(Dataset):
    def __init__(self, x, y,iscuda=False):
        self.X = np.array(x)
        self.y = np.array(y)
        self.cuda = iscuda
    
    def __getitem__(self, index):
        x_val = self.X[index]
        x_val = from_numpy(x_val)
        y_val = from_numpy(np.array([self.y[index]]))
        if self.cuda:
            x_val = x_val.cuda()
            y_val = y_val.cuda()
        return x_val, y_val

    def __len__(self):
        return len(self.X)

    def close(self):
        self.archive.close()

In [ ]:
#use_cuda = cuda.is_available()
train_loader = DataLoader(
                            IrisDataset(X_train, y_train), 
                            batch_size=batch_size_train, 
                            shuffle=True
                        )
test_loader = DataLoader(
                            IrisDataset(X_test, y_test), 
                            batch_size=batch_size_train, 
                            shuffle=True
                        )

## Visualizing the data

In [ ]:
def display_iris(loader, feature_names, target_names):
    
    examples = enumerate(loader)
    batch_idx, (example_data, example_targets) = next(examples)
    for feature_name in feature_names:
        print(feature_name, end="\t")
    print("target")
    for i in range(example_data.shape[0]):
        for j in range(example_data.shape[1]):
            print(float(example_data[i][j]), end="\t\t\t")
        print(target_names[int(example_targets[i]) - 1])

In [ ]:
display_iris(train_loader, feature_names, target_names)

## Building the Network

In [ ]:
class IrisClassifier(nn.Module):

    def __init__(self):
        super(IrisClassifier, self).__init__()
        self.fc1 = nn.Linear(4, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, 3)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, X):
        X = F.relu(self.fc1(X))
        X = self.fc2(X)
        X = self.fc3(X)
        X = self.softmax(X)
        return X

In [ ]:
model = IrisClassifier()
criterion = nn.CrossEntropyLoss()
learning_rate = 0.005
n_epochs = 500
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

## Training the model

In [ ]:
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

In [ ]:
def train(model, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data = Variable(data.float())
        target = Variable(target.reshape(target.shape[0]))
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss))
            train_losses.append(loss.item())
            train_counter.append(
                (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
            save(model.state_dict(), 'results/model.pth')
            save(optimizer.state_dict(), 'results/optimizer.pth')

In [ ]:
def test(model):
    model.eval()
    test_loss = 0
    correct = 0
    with no_grad():
        for data, target in test_loader:
            data = Variable(data.float())
            target = Variable(target.reshape(target.shape[0]))         
            output = model(data)
            test_loss = test_loss + criterion(output, target)
            pred = output.data.max(1, keepdim=True)[1]
            correct = correct + pred.eq(target.data.view_as(pred)).sum()
    test_loss = test_loss / len(test_loader.dataset)
    test_losses.append(test_loss.item())
    print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
try:
    os.mkdir("results")
except OSError:
    print("Results folder exists")
test(model)
for epoch in range(1, n_epochs + 1):
    train(model, epoch)
    test(model)

## Evaluating the model's performance

In [ ]:
fig = plt.figure(figsize=(20,10))
plt.plot(train_counter, train_losses, color='blue')
plt.scatter(test_counter, test_losses, color='red')
plt.legend(['Train Loss', 'Test Loss'], loc='upper right')
plt.xlabel('number of training examples seen')
plt.ylabel('negative log likelihood loss')

## Test prediction

In [ ]:
for batch_idx, (example_data, example_targets) in enumerate(test_loader):

    example_data = Variable(example_data.float())
    example_targets = Variable(example_targets.reshape(example_targets.shape[0]))   
    with no_grad():
        output = model(example_data)
        for i in range(output.shape[0]):
            print("Target = {}, Prediction = {}, ({})".format(
                    example_targets[i], 
                    output.data.max(1, keepdim=True)[1][i].item(),
                    "Match" if example_targets[i] ==  output.data.max(1, keepdim=True)[1][i].item() else "No Match"
                )
            )